# DATA check

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv(('../../../Work/techcross/csv/DATA.csv'))
opt_d_dd = pd.read_csv('../../../Work/techcross/0318/DATA_DEVICE_DATA_OPTIME.csv')

## ANU CHECK

### ANU device의 두 유닛의 각각의 amount의 합을 다시 device별로 합하였을 때, data column의 anu와 일치하는지 여부 파악

In [3]:
#opt_d_dd에서 anu 인덱싱 DEVICE TYPE == 2
anu = opt_d_dd[opt_d_dd['DEVICETYPE'] == 2]
anu = anu.iloc[:, [1, 2, 4, 5, 6, 8, 30, 31, 33, 34, 50, 52, 56, 63, 64, 65, 69]]
anu.shape

(688342, 17)

In [4]:
anu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 688342 entries, 42 to 2128439
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATA_INDEX   688342 non-null  int64  
 1   OP_INDEX     688342 non-null  int64  
 2   DEVICE_ID    688342 non-null  int64  
 3   D_INDEX      688342 non-null  int64  
 4   SHIP_ID      688342 non-null  object 
 5   SECTION      688342 non-null  int64  
 6   ANU_ALARM    688342 non-null  float64
 7   ANU_STATE    688342 non-null  float64
 8   ANU_AMOUNT1  688342 non-null  float64
 9   ANU_AMOUNT2  688342 non-null  float64
 10  DATA_TIME    688342 non-null  object 
 11  STS          688342 non-null  float64
 12  ANU          688342 non-null  float64
 13  LINE         688273 non-null  float64
 14  START_TIME   688273 non-null  object 
 15  END_TIME     688273 non-null  object 
 16  ANU_AMOUNT   688273 non-null  float64
dtypes: float64(8), int64(5), object(4)
memory usage: 94.5+ MB


In [5]:
#SUM 만들어주기
anu['sumamount'] = anu['ANU_AMOUNT1'] + anu['ANU_AMOUNT2']

In [6]:
#그룹핑 후, deviceid별 합 계산
anu_devicegroup = anu.groupby(['SHIP_ID', 'SECTION', 'OP_INDEX', 'DATA_INDEX', 'DATA_TIME', 'ANU'], as_index=False)['sumamount'].sum()
anu_devicegroup

,SHIP_ID,SECTION,OP_INDEX,DATA_INDEX,DATA_TIME,ANU,sumamount
0,T140110-00206,0,44,1,2020-07-26 02:00:49,0.0,0.0
1,T140110-00206,0,44,2,2020-07-26 02:01:49,800.0,800.0
2,T140110-00206,0,45,1,2020-07-26 02:00:49,0.0,0.0
3,T140110-00206,0,45,2,2020-07-26 02:01:49,800.0,800.0
4,T140110-00206,0,46,1,2020-07-26 02:05:05,0.0,0.0
...,...,...,...,...,...,...,...
320454,UnkownShip,2,4,11,2021-06-29 06:01:04,603.0,603.0
320455,UnkownShip,2,4,12,2021-06-29 06:02:04,594.0,593.0
320456,UnkownShip,2,5,1,2021-06-29 06:04:51,1600.0,1600.0
320457,UnkownShip,2,5,2,2021-06-29 06:05:51,1600.0,1600.0


In [7]:
#완전 일치하지는 않는 값 개수(data[ANU]와, ANU[amount_sum])
anu_devicegroup[anu_devicegroup['ANU'] != anu_devicegroup['sumamount']]

,SHIP_ID,SECTION,OP_INDEX,DATA_INDEX,DATA_TIME,ANU,sumamount
93446,T140113-00101,0,20,16,2018-03-24 04:17:16,2138.0,2136.0
93450,T140113-00101,0,20,20,2018-03-24 04:21:16,2375.0,2373.0
93451,T140113-00101,0,29,1,2018-03-24 07:29:12,800.0,798.0
93455,T140113-00101,0,29,5,2018-03-24 07:33:12,905.0,903.0
93457,T140113-00101,0,29,7,2018-03-24 07:35:12,2380.0,2379.0
...,...,...,...,...,...,...,...
320448,UnkownShip,2,4,5,2021-06-29 05:55:04,582.0,573.0
320449,UnkownShip,2,4,6,2021-06-29 05:56:04,606.0,584.0
320451,UnkownShip,2,4,8,2021-06-29 05:58:04,609.0,600.0
320452,UnkownShip,2,4,9,2021-06-29 05:59:04,606.0,600.0


In [8]:
#오차범위 50 포함하지 않는 값 개수
anu_devicegroup[~((anu_devicegroup['ANU'] <= anu_devicegroup['sumamount'] + 50) & (anu_devicegroup['ANU'] >= anu_devicegroup['sumamount'] - 50))]

,SHIP_ID,SECTION,OP_INDEX,DATA_INDEX,DATA_TIME,ANU,sumamount
119470,T140113-00103,0,1,1,2021-05-29 04:26:42,2400.0,2454.0
119471,T140113-00103,0,1,2,2021-05-29 04:27:42,1932.0,2047.0
119472,T140113-00103,0,2,1,2021-05-29 04:31:24,2400.0,2454.0
119473,T140113-00103,0,2,2,2021-05-29 04:32:24,2400.0,2454.0
119474,T140113-00103,0,2,3,2021-05-29 04:33:24,1770.0,1878.0
...,...,...,...,...,...,...,...
320439,UnkownShip,2,2,4,2021-06-30 06:05:22,801.0,3204.0
320440,UnkownShip,2,3,1,2021-06-30 06:02:22,2400.0,4800.0
320441,UnkownShip,2,3,2,2021-06-30 06:03:22,2400.0,4800.0
320442,UnkownShip,2,3,3,2021-06-30 06:04:22,2400.0,3219.0


In [2]:
anu = pd.read_csv(('../../../Work/techcross/csv/ANU.csv'))

In [3]:
anu['SHIP_ID'].unique()

array(['T140110-00206', 'T140113-00101', 'T140113-00102', 'T140113-00103',
       'T140128-00703', 'T140708-00101', 'T141027-00501', 'T160127-00202',
       'T170102-00101', 'T20180208001', 'T20180614001', 'T20180910004',
       'T20181031003', 'T20181218003', 'T20190115004', 'T20190324001',
       'T20190527002', 'T20191014001', 'T20191016008', 'T20191119009',
       'T20200113001', 'T20200218004', 'T20200603001', 'T20201110002',
       'T20201209001', 'T20201210002', 'T20201210003', 'T20201210004',
       'T20201210005', 'T20201210008', 'T20210521011', 'UnkownShip',
       'Unknown'], dtype=object)

In [6]:
data['SHIP_ID'].unique()

array(['T140110-00206', 'T140113-00101', 'T140113-00102', 'T140113-00103',
       'T140113-00104', 'T140128-00703', 'T140703-00101', 'T140708-00101',
       'T160127-00202', 'T170102-00101', 'T20180208001', 'T20180614001',
       'T20180910004', 'T20181031003', 'T20181218003', 'T20190115004',
       'T20190324001', 'T20190527002', 'T20191016008', 'T20191119009',
       'T20200113001', 'T20200218004', 'T20200603001', 'T20201110002',
       'T20201209001', 'T20201210003', 'T20201210004', 'T20201210005',
       'T20210521011', 'Unknown', 'UnkownShip', 'T141027-00501',
       'T20191014001', 'T20201210002', 'T20201210006', 'T20201210008'],
      dtype=object)